In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
prev=pd.read_csv('20170101-20200619_original_fianl.csv')


# In[1585]:


prev.tail()


# In[1571]:


import pandas as pd
#target=점장님이 입력하는 금주 월-금 식수 데이터 
target=pd.DataFrame({'timestamp':['2020-06-15','2020-06-16','2020-06-17','2020-06-22','2020-06-23'],'target_value':[0,1,2,3,4]})


# In[1586]:


#점장님이 입력한 금주 금요일까지 채워지는 함수입니다
def make_df(prev,target):
    import numpy as np
    from datetime import datetime
    from datetime import timedelta
    prev['timestamp']=pd.to_datetime(prev['timestamp'])
    n=(datetime.strptime(target.loc[int(target.index[-1]),'timestamp'],"%Y-%m-%d")-prev.loc[int(prev.index[-1]),'timestamp']).days+1
    
    num=[]
    for i in range(n):
        num.append(int(prev.index[-1]+i))

#     num=[i for i in prev[prev['temperature'].isnull()].index]
#     n=len(num)
    for i in range(n):    
#         prev.loc[num[i]]=np.nan
        prev.loc[num[i],'timestamp']=prev.loc[num[i]-1,'timestamp']+timedelta(days=1)
        prev.loc[num[i],'day_sin']=prev.loc[num[i]-7,'day_sin']
        prev.loc[num[i],'weekday']=prev.loc[num[i]-7,'weekday']
        prev.loc[num[i],'dayofweek_median']=prev.loc[num[i]-7,'dayofweek_median']
 
    url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year=str((prev.loc[prev.index[-1],'timestamp']).year),date=str(prev.loc[prev.index[-1],'timestamp'].month))
    resp=requests.get(url)
    soup=BeautifulSoup(resp.content)
    Text=soup.find_all('locdate')
    holiday=[]
    for i in range(len(Text)):
        holiday.append(pd.to_datetime(Text[i].text))  
    def holiday1(prev):
        if prev['timestamp'] in holiday:
            return 1
        else:
            return 0
    def before_holiday(prev):
        if prev['holiday']==1:
            return 0
        elif prev['timestamp']+timedelta(days=1) in holiday:
            return 1
        else:
            return 0    

    def after_holiday(prev):
        if prev['holiday']==1:
            return 0
        elif prev['timestamp']+timedelta(days=-1) in holiday:
            return 1
        else:
            return 0 
         
    prev['holiday']=prev.apply(holiday1,axis=1)
    prev['before_holiday']=prev.apply(before_holiday,axis=1)
    prev['after_holiday']=prev.apply(after_holiday,axis=1)   
    
# 과거 rainfall
    bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=21&x=18&y=7')  
    soup=BeautifulSoup(bam.text,'html.parser')
    content=soup.find_all('td')
    month=[i.month for i in prev[prev['rainfall'].isnull()]['timestamp']]
    day=[i.day for i in prev[prev['rainfall'].isnull()]['timestamp']]
    prev.loc[prev['rainfall'].isnull(),'rainfall']=[i.replace("\xa0", "0") if i =='\xa0' else i for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]   
    
# 과거 temperature    
    bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=08&x=25&y=4')  
    soup=BeautifulSoup(bam.text,'html.parser')
    content=soup.find_all('td')
    month=[i.month for i in prev[prev['temperature'].isnull()]['timestamp']]
    day=[i.day for i in prev[prev['temperature'].isnull()]['timestamp']]
    prev.loc[prev['temperature'].isnull(),'temperature']=[float(i) for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]
    
# 오늘 강수확률
#     bam=requests.get('https://m.weather.naver.com/')  
#     soup=BeautifulSoup(bam.text,'html.parser')
#     content=soup.find_all('div',{'class':'weather'})
#     prob=[]
#     for i in range(n):
#         prob.append(content[i].find_all('span',{'class':'percent'})[0].text[:-1]) 

#         prev.loc[prev['timestamp']==datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i),'rain_prob']=int(prob[i])*0.1    

# target 주말 0으로 채우기     
    def weekend(prev):
        if prev['weekday'] in [5,6]:
            return 0
        else:
            return prev['target_value']
    prev['target_value']=prev.apply(weekend,axis=1) 
  
    
# 점장님이 입력한 금주 월-목 target 채우기   
    for i in range(len(target)):
        prev.loc[prev['timestamp']==datetime.strptime(target['timestamp'][i],"%Y-%m-%d"),'target_value']=target['target_value'][i]
        
# 기온 이상치     
    def abnormal(prev):
        if (prev['timestamp'].month in [6,7,8]) & (float(prev['temperature'])>=28):
            return 1
        elif (prev['timestamp'].month in [9,10,11]) & (float(prev['temperature'])>=5):
            return 1
        elif (prev['timestamp'].month in [12,1,2]) & (float(prev['temperature'])<=-5):
            return 1
        elif (prev['timestamp'].month in [3,4,5]) & (float(prev['temperature'])<=5):
            return 1
        else:
            return 0
                    
    prev['abnormal']=prev.apply(abnormal,axis=1) 

# 스케일링    
    prev=prev.astype({'temperature' : 'float'})
    prev=prev.astype({'rainfall' : 'float'})
    def z_score_normalize(lst):
        normalized_lst = []
        mean = sum(lst)/len(lst)
        std = np.std(lst)
        normalized_lst = [ (value - mean) / std for value in lst]
        return normalized_lst


    def Scaling(DF):
        # 스케일링할 컬럼 이름
        column_lst = ['rainfall', 'temperature','dayofweek_median','weekday','day_sin']

        for column_name in column_lst:
            DF[column_name] = z_score_normalize(DF[column_name])
        return DF
    
    Scaling(prev)
    return prev


# In[1587]:


import time
start = time.time()
make_df(prev,target).tail(20)


,timestamp,target_value,rainfall,temperature,abnormal,holiday,before_holiday,after_holiday,dayofweek_median,day_sin,menu,menu_nouns,menu_origin,menu_nouns2,scaled_target,weekday,same_menu
1250,2020-06-04,293.0,-0.235915,1.180010,0,0,0,0,0.757595,-0.615532,곤드레밥*양념장,곤드레밥 양념장,NaN,곤드레밥,1.704327,0.000787,1.883931
1251,2020-06-05,141.0,-0.253417,1.374328,0,0,0,0,-0.166729,-1.380648,"뚝배기부대찌개(로스팜:돼지고기-외국산,모듬햄:돼지고기-국내산,소시지:돼지고기-국내산...",뚝배기부대찌개,로스팜 돼지고기 모듬햄 돼지고기 소시지 돼지고기 돈민찌 콩 대두분 인도산 포기김치,부대찌개,1.338942,0.500295,1.112981
1252,2020-06-06,0.0,-0.253417,1.762965,1,0,0,0,-1.491134,-1.107581,NaN,NaN,NaN,NaN,0.000000,0.999804,0.000000
1253,2020-06-07,0.0,-0.253417,1.374328,0,0,0,0,-1.491134,-0.001956,NaN,NaN,NaN,NaN,0.000000,1.499313,0.000000
1254,2020-06-08,373.0,-0.253417,1.762965,1,0,0,0,0.947289,1.103669,"짬뽕밥\n(돼지고기:국내산,오징어:페루산)",짬뽕밥,돼지고기 오징어,짬뽕밥,2.000000,-1.497740,2.000000
1255,2020-06-09,297.0,-0.253417,1.860124,1,0,0,0,0.788636,1.376736,"오삼불고기\n(돼지고기:국내산,오징어:페루산)",오삼불고기,돼지고기 오징어,오삼불고기,1.796247,-0.998231,1.628823
1256,2020-06-10,321.0,0.289137,1.869840,1,0,0,0,0.654126,0.611619,오므라이스,오므라이스,NaN,오므라이스,1.860590,-0.498722,1.910742
1257,2020-06-11,308.0,-0.253417,1.821260,1,0,0,0,0.757595,-0.615532,돼지갈비찜\n(돼지고기:미국산/국내산),돼지갈비찜,돼지고기,돼지갈비,1.825737,0.000787,1.880759
1258,2020-06-12,159.0,-0.253417,1.432624,0,0,0,0,-0.166729,-1.380648,냉모밀,냉모밀,NaN,냉모밀,1.426273,0.500295,1.426273
1259,2020-06-13,0.0,-0.253417,2.005863,1,0,0,0,-1.491134,-1.107581,NaN,NaN,NaN,NaN,0.000000,0.999804,0.000000
